<a href="https://colab.research.google.com/github/BenWheeler99/Capstone-Project/blob/main/GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers torch tkinter pandas


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 22.5 MB/s eta 0:00:00


In [ ]:
import tkinter as tk
from tkinter import scrolledtext
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load the trained model and tokenizer
MODEL_PATH = "/content/drive/MyDrive/Final/book-recommender-model-v3"  # Update with your actual model path
tokenizer = T5Tokenizer.from_pretrained(MODEL_PATH)
model = T5ForConditionalGeneration.from_pretrained(MODEL_PATH)
dataset = pd.read_csv("/content/drive/MyDrive/Final/sampled_dataset_no_nulls_only_EN_NEW.csv")  # Ensure this matches your dataset
model.eval()

# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load FAISS index
index = faiss.IndexFlatL2(embedding_model.get_sentence_embedding_dimension())
index.add(embedding_model.encode(dataset["summary"].tolist(), convert_to_numpy=True))

In [ ]:
def retrieve_top_books():
    query = entry.get()  # Get user input from the text entry

    if not query.strip():
        result_text.delete("1.0", tk.END)
        result_text.insert(tk.END, "Please enter a book description!")
        return

    # Convert query into an embedding
    query_embedding = embedding_model.encode([query], convert_to_numpy=True)

    # Search FAISS index for closest matches
    distances, indices = index.search(query_embedding, 5)  # Get top 5 matches

    # Retrieve book titles and summaries
    results = dataset.iloc[indices[0]][["title", "summary"]]

    # Display results in the GUI
    result_text.delete("1.0", tk.END)
    for idx, row in results.iterrows():
        result_text.insert(tk.END, f"📖 {row['title']}\n{row['summary']}\n\n")


In [ ]:
# Initialize GUI window
root = tk.Tk()
root.title("Book Recommendation System")

# Label
label = tk.Label(root, text="Enter a book description:")
label.pack(pady=5)

# Text entry field
entry = tk.Entry(root, width=60)
entry.pack(pady=5)

# Button to trigger retrieval
search_button = tk.Button(root, text="Find Books", command=retrieve_top_books)
search_button.pack(pady=5)

# Display results
result_text = scrolledtext.ScrolledText(root, width=70, height=15, wrap=tk.WORD)
result_text.pack(pady=10)

# Run the Tkinter main loop
root.mainloop()


TclError: no display name and no $DISPLAY environment variable

In [ ]:
pip install gradio

In [ ]:
import gradio as gr
import faiss
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sentence_transformers import SentenceTransformer

# Load dataset
dataset = pd.read_csv("/content/drive/MyDrive/Final/sampled_dataset_no_nulls_only_EN_NEW.csv")

# Load FAISS index
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
index = faiss.IndexFlatL2(embedding_model.get_sentence_embedding_dimension())
index.add(embedding_model.encode(dataset["summary"].tolist(), convert_to_numpy=True))

# Load your fine-tuned LLM
model_path = "/content/drive/MyDrive/Final/book-recommender-model-v3"  # Update with your saved model path
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Book Recommendation Function
def recommend_books(prompt):
    # Step 1: Embed the input and search FAISS
    query_embedding = embedding_model.encode([prompt], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, 5)  # Retrieve top 5 matches

    # Step 2: Retrieve books from dataset
    retrieved_books = dataset.iloc[indices[0]][["name", "summary"]]

    # Debugging: Ensure books are retrieved
    if retrieved_books.empty:
        return "⚠️ No books found. Try a different description!"

    summaries = retrieved_books["summary"].tolist()

    # Step 3: Generate a recommendation using the LLM
    input_text = f"Recommend a book based on these descriptions: {' '.join(summaries)}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=256).to(device)

    with torch.no_grad():
        output = model.generate(**inputs, max_length=50)

    recommendation = tokenizer.decode(output[0], skip_special_tokens=True)

    # Step 4: Format the book list output
    book_list = "\n\n".join([f"📖 **{row['name']}**\n{row['summary']}" for _, row in retrieved_books.iterrows()])

    return f"🤖 **AI Recommendation:** {recommendation}\n\n🔎 **Top Matches:**\n\n{book_list}"

# Create a Gradio interface with a dedicated output box
interface = gr.Interface(
    fn=recommend_books,
    inputs=gr.Textbox(label="Enter a book description", placeholder="Describe the type of book you're looking for..."),
    outputs=gr.Textbox(label="Recommended Books", lines=10, interactive=False),  # Dedicated output box
    title="📚 AI-Powered Book Recommendation System",
    description="Enter a book description and get AI-generated recommendations based on similar books!",
)

# Launch the app
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://df63a3f25ef16ba656.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
